In [96]:
#Imports and API Key
#building in offsets

#checkout scikit learn timeseries 
#checkout cross validation - rolling window

#Imports and API Key

import pandas as pd
import quandl
from scipy import stats
import scipy
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

from sklearn.pipeline import Pipeline

from basis_expansions.basis_expansions import (
    Polynomial, LinearSpline)

from regression_tools.dftransformers import (
    ColumnSelector, Identity, FeatureUnion, MapFeature, Intercept)

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeClassifier

from sklearn.model_selection import TimeSeriesSplit

import xgboost as xgb

%aimport dataclean

quandl.ApiConfig.api_key = 'm8FYMyoCaJSbTrBASNHh'

In [97]:
#pulling data from Quandl

data = pd.read_csv('datasources2.csv') #staging the QUANDL keys to pull in CSV
cols = list(data['Var_name'].astype('str'))
dataset = quandl.get([val for val in data['Quandl Key']]) #looping through the QUANDL keys to pull it into one DF
dataset.columns = cols

In [98]:
#pulling FED Yield Curve data

yields = pd.read_csv('Fed10Y_3M.csv')
yields['Date'] = pd.to_datetime(yields['Date'])
yields['Date'] = yields['Date'].apply(lambda x: x.strftime('%Y-%m'))
yields = yields.set_index('Date')
yields = yields.drop(['3 Month Treasury Yield', 'Rec_prob', 'NBER_Rec','Unnamed: 7'], axis=1) 

In [99]:
#need to write in special logic to factor in for 3YRT being a daily

treas = dataset['3YRT']
dataset = dataset.drop(columns = ['3YRT'])

In [100]:
treas = treas.resample('MS').mean()

In [101]:
## back to working on the general data
dataset.index = dataset.index.strftime('%Y-%m') #converting the datetime index to Y/M so it is collapsable
dataset = dataset.groupby(dataset.index, as_index=True).agg(sum) #collapsing by Y/M

In [102]:
#readd 3YRT back into data

dataset = dataset.join(treas, how='outer')

In [103]:
#converting GDP quarterly data into monthly - need to convert it so it fills in the following 3 months

dataset = dataclean.convert_q_to_m(dataset, 'GDP')

#converting consumer sentiment into monthly

dataset = dataclean.convert_q_to_m(dataset, 'CONS_SENT')

In [106]:
offset_dict = {'PMI':1, 'UNR':1, 'YUNR':1,'US_NHOME_SALES':1, 'PART_TIME':1, 'CPPR_PRICE':1, 'HOUS_PERMS':1, 'HOUS_STARTS':1, 'CAP_UTIL':1, 'PERS_SAVINGS':3, 'EXPORTS':3, 'IMPORTS':3, 'TRADE_BALANCE':3, 'US_M2':1, 'US_NHOME_SALES':1,'PPI':1,'CPI':1} #positives shift down, negatives shift up. Goal of this dict is to align data to when it gets released

In [107]:
for entry in offset_dict:
    #print(entry)
    #print(offset_dict[entry])
    dataset[str(entry)] = dataset[str(entry)].shift(offset_dict[entry])

In [108]:
#calculating change in GDP and converting Y into categorical values 
dataset['Recession'] = ((dataset['GDP'] - dataset['GDP'].shift(3)) < 0).astype(int)
dataset = dataset.drop(columns = ['GDP']) #dropping calc column and recession column from dataset, experimenting with taking out fed funds rate

In [109]:
#merge fed interest rate data here
dataset = dataset.join(yields, how='outer')

In [110]:
dataset.shape

(1286, 23)

In [111]:
dataset = dataset[552:]

In [112]:
dataset = dataset[:-12]

In [113]:
#substituting mean value in for missing values and adding dummy column to indicate where done.

#this could be leakage, use backwards looking means

for col in dataset.columns:
    if str(col)=='Recession':
        continue
    dataclean.clean_zeros(col, dataset)

In [114]:
dataset['3YRT'] = dataset['3YRT'].fillna(dataset['3YRT'].mean())

In [115]:
dataset.head()

,PMI,UNR,YUNR,CONS_SENT,PART_TIME,CPPR_PRICE,HOUS_PERMS,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,...,3 Month Treasury Yield (Bond Equivalent Basis),Spread,CPPR_PRICE_PXY,HOUS_PERMS_PXY,HOUS_STARTS_PXY,CAP_UTIL_PXY,PERS_SAVINGS_PXY,US_M2_PXY,US_NHOME_SALES_PXY,CPI_PXY
1959-01-01,62.700,6.200,12.000,90.800,1081.000,165.089,1331.222,1429.532,69.480,8.779,...,2.880,1.140,1,1,1,1,1,1,1,0
1959-02-01,60.500,6.200,12.100,90.800,1022.000,165.089,1331.222,1657.000,69.480,8.779,...,2.760,1.200,1,1,0,1,1,0,1,0
1959-03-01,64.400,6.000,11.600,90.800,973.000,165.089,1331.222,1667.000,69.480,8.779,...,2.860,1.130,1,1,0,1,1,0,1,0
1959-04-01,66.900,5.900,11.100,90.800,1102.000,165.089,1331.222,1620.000,69.480,11.300,...,3.010,1.110,1,1,0,1,0,0,1,0
1959-05-01,67.100,5.600,11.100,95.300,1086.000,165.089,1331.222,1590.000,69.480,10.600,...,2.900,1.410,1,1,0,1,0,0,1,0


In [116]:
#adding momentum factors

momentum_cols = list(dataset.columns[:-6])

momentum_cols.remove('PPI') #removing PPI and CPI because they need a different transformation
momentum_cols.remove('CPI')
momentum_cols.remove('Recession')

for i in [1,3,12]:
    for col in momentum_cols:
        if 'PXY' in str(col): #adding logic so it doesn't create a momentum column out of PXY columns
            continue
        dataclean.create_momentum(col,dataset,i)

In [117]:
dataset.head()

,PMI,UNR,YUNR,CONS_SENT,PART_TIME,CPPR_PRICE,HOUS_PERMS,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,...,EXPORTS_12m_shift,IMPORTS_12m_shift,TRADE_BALANCE_12m_shift,INT_RATE_12m_shift,US_M2_12m_shift,US_NHOME_SALES_12m_shift,3YRT_12m_shift,10 Year Treasury Yield_12m_shift,3 Month Treasury Yield (Bond Equivalent Basis)_12m_shift,Spread_12m_shift
1959-01-01,62.700,6.200,12.000,90.800,1081.000,165.089,1331.222,1429.532,69.480,8.779,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1959-02-01,60.500,6.200,12.100,90.800,1022.000,165.089,1331.222,1657.000,69.480,8.779,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1959-03-01,64.400,6.000,11.600,90.800,973.000,165.089,1331.222,1667.000,69.480,8.779,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1959-04-01,66.900,5.900,11.100,90.800,1102.000,165.089,1331.222,1620.000,69.480,11.300,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1959-05-01,67.100,5.600,11.100,95.300,1086.000,165.089,1331.222,1590.000,69.480,10.600,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [118]:
#CPI Calcs

for i in [1,3,12]:
    for col in ['CPI','PPI']:
        dataclean.infl_momentum(col,dataset,i)

In [119]:
dataset.shape

(722, 97)

In [120]:
#cutoff most of missing data, Post March 2019, Prior 1959. CPI/PPI missing 2016 onward so need to cut that off
#dataset = dataset.iloc[552:]
#dataset = dataset.iloc[:-2]

#y = y.iloc[552:]
y = dataset['Recession']
dataset = dataset.drop(columns = ['Recession'])
X = dataset

In [121]:
### Data Prep Finished Here ###

In [143]:
y_shift = y.shift(-12) #needs to be negative to look forward
y_shift = y_shift.fillna(0)

In [144]:
X_train = X.iloc[12:550]
X_test = X.iloc[550:]
y_train = y_shift.iloc[12:550]
y_test = y_shift.iloc[550:]

In [145]:
model = xgb.XGBClassifier(learning_rate=0.005,
                                   n_estimators=1000, #bump this and learning rate to make more trees, trees(not exact) = n*learning rate
                                   min_samples_leaf=4,
                                   max_depth=1,
                                   subsample=0.5)

model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.005, max_delta_step=0,
       max_depth=1, min_child_weight=1, min_samples_leaf=4, missing=None,
       n_estimators=1000, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.5)

In [146]:
probs = model.predict_proba(X_test)

In [147]:
log_loss(y_test, probs)

0.42980606562652907

In [148]:
roc_auc_score(y_test.values, probs[:,1:])

0.7198761261261261

In [149]:
results = pd.DataFrame(probs)
results['actual'] = y_test.values
results.index = y_test.index
pd.set_option('display.float_format', lambda x: '%.3f' % x)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(results)

               0     1  actual
2004-11-01 0.948 0.052   0.000
2004-12-01 0.956 0.044   0.000
2005-01-01 0.953 0.047   0.000
2005-02-01 0.946 0.054   0.000
2005-03-01 0.942 0.058   0.000
2005-04-01 0.953 0.047   0.000
2005-05-01 0.941 0.059   0.000
2005-06-01 0.952 0.048   0.000
2005-07-01 0.918 0.082   0.000
2005-08-01 0.910 0.090   0.000
2005-09-01 0.907 0.093   0.000
2005-10-01 0.885 0.115   0.000
2005-11-01 0.914 0.086   0.000
2005-12-01 0.895 0.105   0.000
2006-01-01 0.843 0.157   0.000
2006-02-01 0.714 0.286   0.000
2006-03-01 0.766 0.234   0.000
2006-04-01 0.887 0.113   0.000
2006-05-01 0.887 0.113   0.000
2006-06-01 0.892 0.108   0.000
2006-07-01 0.819 0.181   0.000
2006-08-01 0.706 0.294   0.000
2006-09-01 0.742 0.258   0.000
2006-10-01 0.837 0.163   0.000
2006-11-01 0.807 0.193   0.000
2006-12-01 0.833 0.167   0.000
2007-01-01 0.820 0.180   1.000
2007-02-01 0.813 0.187   1.000
2007-03-01 0.775 0.225   1.000
2007-04-01 0.659 0.341   0.000
2007-05-01 0.757 0.243   0.000
2007-06-

In [150]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(y)

1959-01-01   0.000
1959-02-01   0.000
1959-03-01   0.000
1959-04-01   0.000
1959-05-01   0.000
1959-06-01   0.000
1959-07-01   0.000
1959-08-01   0.000
1959-09-01   0.000
1959-10-01   0.000
1959-11-01   0.000
1959-12-01   0.000
1960-01-01   0.000
1960-02-01   0.000
1960-03-01   0.000
1960-04-01   1.000
1960-05-01   1.000
1960-06-01   1.000
1960-07-01   0.000
1960-08-01   0.000
1960-09-01   0.000
1960-10-01   1.000
1960-11-01   1.000
1960-12-01   1.000
1961-01-01   0.000
1961-02-01   0.000
1961-03-01   0.000
1961-04-01   0.000
1961-05-01   0.000
1961-06-01   0.000
1961-07-01   0.000
1961-08-01   0.000
1961-09-01   0.000
1961-10-01   0.000
1961-11-01   0.000
1961-12-01   0.000
1962-01-01   0.000
1962-02-01   0.000
1962-03-01   0.000
1962-04-01   0.000
1962-05-01   0.000
1962-06-01   0.000
1962-07-01   0.000
1962-08-01   0.000
1962-09-01   0.000
1962-10-01   0.000
1962-11-01   0.000
1962-12-01   0.000
1963-01-01   0.000
1963-02-01   0.000
1963-03-01   0.000
1963-04-01   0.000
1963-05-01  

In [130]:
print(sum(y))

84.0


In [69]:
X_train

,PMI,UNR,YUNR,CONS_SENT,HOME_SALES,PART_TIME,CPPR_PRICE,HOUS_PERMS,HOUS_STARTS,CAP_UTIL,...,3YRT_12m_shift,10 Year Treasury Yield_12m_shift,3 Month Treasury Yield (Bond Equivalent Basis)_12m_shift,Spread_12m_shift,CPI_1m_shift,PPI_1m_shift,CPI_3m_shift,PPI_3m_shift,CPI_12m_shift,PPI_12m_shift
1960-01-01,58.200,5.300,11.100,93.800,3353.324,1000.000,165.089,1331.222,1601.000,69.480,...,0.000,0.700,1.580,-0.880,0.000,0.000,0.341,-0.631,1.730,-0.316
1960-02-01,61.500,5.200,10.900,93.800,3353.324,1015.000,165.089,1092.000,1460.000,69.480,...,0.000,0.530,1.300,-0.770,-0.340,0.317,-0.340,0.000,1.034,-0.315
1960-03-01,52.300,4.800,10.200,93.800,3353.324,1062.000,165.089,1088.000,1503.000,69.480,...,0.000,0.260,0.520,-0.260,0.341,0.000,0.000,0.317,1.730,-0.315
1960-04-01,47.800,5.400,11.500,93.800,3353.324,888.000,165.089,955.000,1109.000,69.480,...,0.000,0.160,0.290,-0.130,0.000,0.633,0.000,0.952,1.730,0.315
1960-05-01,45.300,5.200,10.900,93.300,3353.324,1041.000,165.089,1016.000,1289.000,69.480,...,0.000,0.040,0.460,-0.420,0.340,0.000,0.683,0.633,1.724,0.000
1960-06-01,42.600,5.100,10.700,93.300,3353.324,988.000,165.089,1052.000,1271.000,69.480,...,0.000,-0.190,-0.770,0.580,0.000,-0.314,0.340,0.316,1.724,-0.314
1960-07-01,44.400,5.400,11.000,93.300,3353.324,966.000,165.089,958.000,1247.000,69.480,...,0.000,-0.500,-0.920,0.420,0.339,0.000,0.680,-0.314,1.718,0.000
1960-08-01,43.700,5.500,10.800,97.200,3353.324,1013.000,165.089,999.000,1197.000,69.480,...,0.000,-0.630,-1.110,0.480,0.000,0.000,0.339,-0.314,1.370,0.000
1960-09-01,47.600,5.600,11.400,97.200,3353.324,1018.000,165.089,994.000,1344.000,69.480,...,0.000,-0.880,-1.610,0.730,0.000,-0.315,0.339,-0.315,1.370,0.000
1960-10-01,45.400,5.500,11.000,97.200,3353.324,1027.000,165.089,984.000,1097.000,69.480,...,0.000,-0.640,-1.800,1.160,0.000,0.000,0.000,-0.315,1.024,-0.315


In [70]:
model.feature_importances_

array([0.02859668, 0.        , 0.02116571, 0.01599991, 0.        ,
       0.        , 0.0163054 , 0.02966032, 0.01813553, 0.02364324,
       0.        , 0.02624166, 0.        , 0.        , 0.00735066,
       0.02983808, 0.        , 0.01197086, 0.        , 0.01914044,
       0.02059533, 0.02373335, 0.01573316, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01812162, 0.        ,
       0.02036607, 0.01344542, 0.0206742 , 0.        , 0.        ,
       0.01428413, 0.01937304, 0.01821684, 0.02002154, 0.01928606,
       0.01551074, 0.        , 0.        , 0.        , 0.        ,
       0.01740243, 0.        , 0.        , 0.01672828, 0.02212291,
       0.02486654, 0.01810698, 0.        , 0.01065295, 0.        ,
       0.02111405, 0.        , 0.01905129, 0.00199334, 0.02367065,
       0.02297047, 0.        , 0.        , 0.00948641, 0.        ,
       0.        , 0.        , 0.        , 0.01308319, 0.     

In [71]:
for combo in zip(X_train.columns,model.feature_importances_):
    print(combo)

('PMI', 0.028596684)
('UNR', 0.0)
('YUNR', 0.021165708)
('CONS_SENT', 0.015999908)
('HOME_SALES', 0.0)
('PART_TIME', 0.0)
('CPPR_PRICE', 0.016305402)
('HOUS_PERMS', 0.02966032)
('HOUS_STARTS', 0.018135529)
('CAP_UTIL', 0.023643237)
('PERS_SAVINGS', 0.0)
('EXPORTS', 0.02624166)
('IMPORTS', 0.0)
('TRADE_BALANCE', 0.0)
('INT_RATE', 0.007350656)
('US_M2', 0.029838076)
('US_NHOME_SALES', 0.0)
('PPI', 0.01197086)
('CPI', 0.0)
('3YRT', 0.019140435)
('10 Year Treasury Yield', 0.02059533)
('3 Month Treasury Yield (Bond Equivalent Basis)', 0.023733353)
('Spread', 0.015733158)
('HOME_SALES_PXY', 0.0)
('CPPR_PRICE_PXY', 0.0)
('HOUS_PERMS_PXY', 0.0)
('HOUS_STARTS_PXY', 0.0)
('CAP_UTIL_PXY', 0.0)
('PERS_SAVINGS_PXY', 0.0)
('US_M2_PXY', 0.0)
('US_NHOME_SALES_PXY', 0.0)
('CPI_PXY', 0.0)
('PMI_1m_shift', 0.0)
('UNR_1m_shift', 0.018121617)
('YUNR_1m_shift', 0.0)
('CONS_SENT_1m_shift', 0.020366065)
('HOME_SALES_1m_shift', 0.013445417)
('PART_TIME_1m_shift', 0.020674195)
('CPPR_PRICE_1m_shift', 0.0)
('HOU

In [72]:
importance = pd.DataFrame(zip(X_train.columns,model.feature_importances_), columns = ['Indicator','Importance'])

In [73]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(importance.sort_values(['Importance'], ascending = False))

                                             Indicator  Importance
15                                               US_M2       0.030
7                                           HOUS_PERMS       0.030
0                                                  PMI       0.029
94                                    Spread_12m_shift       0.027
81                                HOUS_PERMS_12m_shift       0.027
11                                             EXPORTS       0.026
100                                      PPI_12m_shift       0.026
77                                 CONS_SENT_12m_shift       0.026
55                                       YUNR_3m_shift       0.025
78                                HOME_SALES_12m_shift       0.024
21      3 Month Treasury Yield (Bond Equivalent Basis)       0.024
64                                    EXPORTS_3m_shift       0.024
9                                             CAP_UTIL       0.024
65                                    IMPORTS_3m_shift       0

In [74]:
X_train.head(100)

,PMI,UNR,YUNR,CONS_SENT,HOME_SALES,PART_TIME,CPPR_PRICE,HOUS_PERMS,HOUS_STARTS,CAP_UTIL,...,3YRT_12m_shift,10 Year Treasury Yield_12m_shift,3 Month Treasury Yield (Bond Equivalent Basis)_12m_shift,Spread_12m_shift,CPI_1m_shift,PPI_1m_shift,CPI_3m_shift,PPI_3m_shift,CPI_12m_shift,PPI_12m_shift
1960-01-01,58.200,5.300,11.100,93.800,3353.324,1000.000,165.089,1331.222,1601.000,69.480,...,0.000,0.700,1.580,-0.880,0.000,0.000,0.341,-0.631,1.730,-0.316
1960-02-01,61.500,5.200,10.900,93.800,3353.324,1015.000,165.089,1092.000,1460.000,69.480,...,0.000,0.530,1.300,-0.770,-0.340,0.317,-0.340,0.000,1.034,-0.315
1960-03-01,52.300,4.800,10.200,93.800,3353.324,1062.000,165.089,1088.000,1503.000,69.480,...,0.000,0.260,0.520,-0.260,0.341,0.000,0.000,0.317,1.730,-0.315
1960-04-01,47.800,5.400,11.500,93.800,3353.324,888.000,165.089,955.000,1109.000,69.480,...,0.000,0.160,0.290,-0.130,0.000,0.633,0.000,0.952,1.730,0.315
1960-05-01,45.300,5.200,10.900,93.300,3353.324,1041.000,165.089,1016.000,1289.000,69.480,...,0.000,0.040,0.460,-0.420,0.340,0.000,0.683,0.633,1.724,0.000
1960-06-01,42.600,5.100,10.700,93.300,3353.324,988.000,165.089,1052.000,1271.000,69.480,...,0.000,-0.190,-0.770,0.580,0.000,-0.314,0.340,0.316,1.724,-0.314
1960-07-01,44.400,5.400,11.000,93.300,3353.324,966.000,165.089,958.000,1247.000,69.480,...,0.000,-0.500,-0.920,0.420,0.339,0.000,0.680,-0.314,1.718,0.000
1960-08-01,43.700,5.500,10.800,97.200,3353.324,1013.000,165.089,999.000,1197.000,69.480,...,0.000,-0.630,-1.110,0.480,0.000,0.000,0.339,-0.314,1.370,0.000
1960-09-01,47.600,5.600,11.400,97.200,3353.324,1018.000,165.089,994.000,1344.000,69.480,...,0.000,-0.880,-1.610,0.730,0.000,-0.315,0.339,-0.315,1.370,0.000
1960-10-01,45.400,5.500,11.000,97.200,3353.324,1027.000,165.089,984.000,1097.000,69.480,...,0.000,-0.640,-1.800,1.160,0.000,0.000,0.000,-0.315,1.024,-0.315


In [80]:
X = X.iloc[12:]
y = y.iloc[12:]

In [89]:
tscv = TimeSeriesSplit(n_splits=4)

In [90]:
TimeSeriesSplit(max_train_size=None, n_splits=4)

TimeSeriesSplit(max_train_size=None, n_splits=4)

In [95]:
for train_index, test_index in tscv.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #print(X_train, y_train)
    
    
    model = xgb.XGBClassifier(learning_rate=0.005,
                                   n_estimators=1000, #bump this and learning rate to make more trees, trees(not exact) = n*learning rate
                                   min_samples_leaf=4,
                                   max_depth=1,
                                   subsample=0.5)

    model.fit(X_train, y_train)
    
    probs = model.predict_proba(X_test)
    
    #print(probs)
    
    print(log_loss(y_test, probs))
    
    print(roc_auc_score(y_test.values, probs[:,1:]))
    
    results = pd.DataFrame(probs)
    results['actual'] = y_test.values
    results.index = y_test.index
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(results)
    

0.10823615615479124
0.7314148681055155
               0     1  actual
1971-11-01 0.985 0.015   0.000
1971-12-01 0.967 0.033   0.000
1972-01-01 0.987 0.013   0.000
1972-02-01 0.987 0.013   0.000
1972-03-01 0.979 0.021   0.000
1972-04-01 0.973 0.027   0.000
1972-05-01 0.959 0.041   0.000
1972-06-01 0.966 0.034   0.000
1972-07-01 0.973 0.027   0.000
1972-08-01 0.979 0.021   0.000
1972-09-01 0.988 0.012   0.000
1972-10-01 0.975 0.025   0.000
1972-11-01 0.977 0.023   0.000
1972-12-01 0.989 0.011   0.000
1973-01-01 0.969 0.031   0.000
1973-02-01 0.975 0.025   0.000
1973-03-01 0.958 0.042   0.000
1973-04-01 0.972 0.028   0.000
1973-05-01 0.951 0.049   0.000
1973-06-01 0.968 0.032   0.000
1973-07-01 0.953 0.047   0.000
1973-08-01 0.950 0.050   0.000
1973-09-01 0.935 0.065   0.000
1973-10-01 0.926 0.074   0.000
1973-11-01 0.933 0.067   0.000
1973-12-01 0.936 0.064   0.000
1974-01-01 0.924 0.076   0.000
1974-02-01 0.917 0.083   0.000
1974-03-01 0.940 0.060   0.000
1974-04-01 0.953 0.047   0.000


0.0906114569261775
0.8872901678657075
               0     1  actual
1995-07-01 0.985 0.015   0.000
1995-08-01 0.991 0.009   0.000
1995-09-01 0.988 0.012   0.000
1995-10-01 0.991 0.009   0.000
1995-11-01 0.989 0.011   0.000
1995-12-01 0.987 0.013   0.000
1996-01-01 0.990 0.010   0.000
1996-02-01 0.980 0.020   0.000
1996-03-01 0.985 0.015   0.000
1996-04-01 0.988 0.012   0.000
1996-05-01 0.990 0.010   0.000
1996-06-01 0.988 0.012   0.000
1996-07-01 0.990 0.010   0.000
1996-08-01 0.991 0.009   0.000
1996-09-01 0.992 0.008   0.000
1996-10-01 0.991 0.009   0.000
1996-11-01 0.991 0.009   0.000
1996-12-01 0.990 0.010   0.000
1997-01-01 0.992 0.008   0.000
1997-02-01 0.992 0.008   0.000
1997-03-01 0.991 0.009   0.000
1997-04-01 0.988 0.012   0.000
1997-05-01 0.991 0.009   0.000
1997-06-01 0.990 0.010   0.000
1997-07-01 0.990 0.010   0.000
1997-08-01 0.991 0.009   0.000
1997-09-01 0.992 0.008   0.000
1997-10-01 0.992 0.008   0.000
1997-11-01 0.991 0.009   0.000
1997-12-01 0.990 0.010   0.000
1

In [88]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(y)

1960-01-01   0.000
1960-02-01   0.000
1960-03-01   0.000
1960-04-01   1.000
1960-05-01   1.000
1960-06-01   1.000
1960-07-01   0.000
1960-08-01   0.000
1960-09-01   0.000
1960-10-01   1.000
1960-11-01   1.000
1960-12-01   1.000
1961-01-01   0.000
1961-02-01   0.000
1961-03-01   0.000
1961-04-01   0.000
1961-05-01   0.000
1961-06-01   0.000
1961-07-01   0.000
1961-08-01   0.000
1961-09-01   0.000
1961-10-01   0.000
1961-11-01   0.000
1961-12-01   0.000
1962-01-01   0.000
1962-02-01   0.000
1962-03-01   0.000
1962-04-01   0.000
1962-05-01   0.000
1962-06-01   0.000
1962-07-01   0.000
1962-08-01   0.000
1962-09-01   0.000
1962-10-01   0.000
1962-11-01   0.000
1962-12-01   0.000
1963-01-01   0.000
1963-02-01   0.000
1963-03-01   0.000
1963-04-01   0.000
1963-05-01   0.000
1963-06-01   0.000
1963-07-01   0.000
1963-08-01   0.000
1963-09-01   0.000
1963-10-01   0.000
1963-11-01   0.000
1963-12-01   0.000
1964-01-01   0.000
1964-02-01   0.000
1964-03-01   0.000
1964-04-01   0.000
1964-05-01  